In [2]:
import os
from pypcd import pypcd
import numpy as np

In [3]:
def save_pcloud(data: np.ndarray, filename: str):
    pcloud = pypcd.make_xyz_point_cloud(data)
    pypcd.save_point_cloud_bin(pcloud, f"{filename}.pcd")

In [4]:
def quantize(data: np.ndarray, steps: np.ndarray) -> np.ndarray:
    data = np.around(data/steps)
    data = np.unique(data, axis=0)
    return data

In [5]:
a = np.array([1])
a[:, None].shape

(1, 1)

In [6]:
def pts_around_lidar(data: np.ndarray, cube: list()) -> np.ndarray:
    x_ind = np.where(cube[0] < data[:, 0], 1, 0) * \
        np.where(data[:, 0] < cube[1], 1, 0)
    y_ind = np.where(cube[2] < data[:, 1], 1, 0) * \
        np.where(data[:, 1] < cube[3], 1, 0)
    z_ind = np.where(cube[4] < data[:, 2], 1, 0) * \
        np.where(data[:, 2] < cube[5], 1, 0)
    cube_ind = x_ind*y_ind*z_ind
    data = data - data*cube_ind[:, None]
    return np.unique(data, axis=0)

In [7]:
dpath = '/home/mohit/webots_code/data'
lidar_path = os.path.join(dpath, 'lidar_samples')
label_path = os.path.join(dpath, 'sample_label')
gps_path = os.path.join(dpath, 'samples')

In [16]:
lidar_data = np.load(os.path.join(lidar_path, '25.npy'))

In [17]:
print(lidar_data.shape, lidar_data.dtype)

(222402, 3) float32


In [18]:
boundaries = {'xmax': 120, 'xmin': -120,
              'zmax': 120, 'zmin': -120, "ymin": -5, 'ymax': 5}
steps = np.array([1.0, 0.5, 1.0])
cube = [-1, 1, -5, 5, -5, 5]

In [20]:
lidar_un = quantize(lidar_data, steps)
lidar_tr = pts_around_lidar(lidar_data, cube)
print(lidar_un.shape)
print(lidar_tr.shape)
save_pcloud(lidar_data, 'raw')
save_pcloud(lidar_un, 'quant')
save_pcloud(lidar_tr, 'trunc')
lidar_t_un = pts_around_lidar(lidar_un, cube)
save_pcloud(lidar_t_un, 'final')

(6433, 3)
(207335, 3)
